In [51]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_ps.csv', sep=',')
display(melb_df['Address'].loc[133])
display(melb_df['Address'].loc[244])
display(melb_df['Address'].loc[4555])

'2c Hamilton St'

'8/58 Sutherland Rd'

'8 Riley St'

In [52]:
def get_street_type(addres):
    #список географических пометок в адресе
    exclude_list = ['N', 'S', 'W', 'E']
    #получаем список слов в строке и заносим его в список слов
    address_list = addres.split(' ')
    #обрезаем список, оставляем последний элемент
    street_type = address_list[-1]
    #делаем проверку
    if street_type in exclude_list:
        #если street_type является пометкой, то переопределяем её на второй элемент с конца списка address_list
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)
    
popylar_stypes = street_types.value_counts().nlargest(10).index

#Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, 
# которая будет проверять, есть ли строка x в этом перечне, и, если это так, 
# lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. 
# Наконец, применим такую функцию к Series street_types, полученной ранее,
# а результат определим в новый столбец таблицы StreetTyp

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popylar_stypes else 'other')
display(melb_df['StreetType'])
melb_df = melb_df.drop('Address', axis=1)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [46]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.day_of_week
#Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи.
# Из полученных в задании результатов можно сделать вывод,
# что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).
# Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1,
# если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.
# Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, 
# проданного в выходные дни.

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
     
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean(), 2))

1081198.64

In [56]:
#Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: 
# оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', 
# больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. 

popylar_seller = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popylar_seller else 'other')
a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))


1.3


In [57]:
# Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. 
# В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. 
# В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. 
# Структура текста в столбце фиксирована и не может измениться.
# Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
# Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
# Примечание. Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. 
# Учтите это при построении своей функции.

import pandas as pd
test_series_1 = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])

test_series_2 = pd.Series([
    'Опыт работы 5 лет',
    'Опыт работы 5 месяцев',
    'Опыт работы 1 год 1 месяц',
    'Опыт работы 3 месяца',
    'Опыт работы 7 лет'
])

def get_experience(arg):
    month_key_words = ['месяц', "месяцев", "месяца"]
    year_key_words = ["год", "лет", "года"]
    args_slited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_slited)):
        if args_slited[i] in month_key_words:
            month = args_slited[i-1]
        if args_slited[i] in year_key_words:
            year = args_slited[i-1]
    return int(year)*12 + int(month)       

print(test_series_1.apply(get_experience))
            

0    99
1    41
2    21
3     3
4    72
dtype: int64
